In [1]:
import ivolat3 as iv
import pandas as pd
import numpy as np
from bokeh.io import output_notebook
from bokeh.plotting import figure, show
from bokeh.models import HoverTool
output_notebook()

Loading BokehJS ...

## ポジションのデルタが変化するということを示す
* 上がると思ってリスクを取った場合  
* 先物のみ、コールオプションのみ、コール＋デルタヘッジを見る

In [2]:
s = 18000
k = 19000
r = 0.001
q = 0
t = 30/365
sigma = 0.2

先物のキャッシュフロー : 購入価格　±　現値

In [3]:
future = {}

for i in range(14000,22000):
    fut = i-s
    future[i] = fut

In [4]:
f = pd.Series(future)

In [5]:
hover = HoverTool(
            tooltips=[
                ('(x,y)','(@x,@y)')
            ]
                )

p = figure(title='futures PL', plot_width = 500, plot_height=300,tools=[hover,'crosshair'])
p.line(x=f.index, y=f)

show(p)

* 先物の損益はきれいな直線となります。
* 日経平均先物はラージがデルタ１、ミニがデルタ0.1
* 次にオプションの損益を見てみます

In [6]:
call = {}

for i in range(14000,22000):
    prem = iv.prem_call(i,k,r,q,t,sigma)
    call[i] = prem
    
c = pd.Series(call)

In [7]:
hover = HoverTool(
            tooltips=[
                ('(x,y)','(@x,@y)')
            ]
                )

p = figure(title='c19000 prem', plot_width=500, plot_height=300, tools=[hover])
p.line(x=c.index, y=c)

show(p)

* 18000円の時の価値を調べると98.6円なので、100円＊1000の10万円で残り30日のコールが買えることが分かります。

In [8]:
c[18000] # 18000円の時のコールの値段

98.601234229973215

In [9]:
c_delta = iv.delta_call(s,k,r,q,t,sigma)
c_delta

0.18064583391544065

* デルタを見ると0.18となっている。日経平均先物1枚分だと、大体5－6枚コールを買わなければいけない。
* というわけで5枚買った時の曲線を見てみる

In [10]:
call_2 = {}

for i in range(14000, 22000):
    prem = 5*iv.prem_call(i,k,r,q,t,sigma)
    call_2[i] = prem

In [11]:
c2 = pd.Series(call_2)

In [12]:
hover = HoverTool(
            tooltips=[
                ('(x,y)','(@x,@y)')
            ]
                )

p = figure(plot_width=500,plot_height=300, title='c19000 * 5',tools=[hover])
p.line(x=c2.index,y=c2)

show(p)

In [13]:
c2[19000]

2176.6199232553754

* 先物だったら株価が19,000円になっても1,000＊1,000で100万円しか儲からないのに、オプションだと同じリスクでも2176*1000で200万超える
## まぁ時間が経ってなかったらな

In [14]:
call_3 = {}

for i in range(14000,22000):
    prem = iv.prem_call(i,k,r,q,10/365,sigma)
    call_3[i] = prem
    
c3 = pd.Series(call_3)

In [15]:
hover = HoverTool(
            tooltips=[
                ('(x,y)','(@x,@y)')
            ]
                )

p = figure(title='c18000 10days', plot_width=500, plot_height=300, tools=[hover])
p.line(c3.index,c3)

show(p)

In [16]:
c3[19000] - c[18000]

152.57247579142449

* 残り10日になると日経平均株価が19,000円になっても152×1,000　＝15万しか儲からんです。

In [17]:
c3[18000] - c[18000]

-85.42610218143875

* 残り10日になっても同じ値段だったら-85×1000＝－85000損します。5枚買ってたら42万5千円損。
* 先物なら同じ値段の場合±0円です。
* でも日経平均株価が14,000円になったら先物だったら400万損するけど、オプションの買いは1枚10万円しかやられない。

## デルタをニュートラルにしてみる
* c18000円と先物の売りを組み合わせてみる
* c18000のデルタは0.18なので、先物を0.18枚売って仮想損益図を作る。

In [18]:
gou = {}

for i in range(14000, 22000):
    call = iv.prem_call(i,k,r,q,t,sigma)
    fut = 0.18*(s- i)
    posi = call + fut
    
    gou[i] = posi
    
g = pd.Series(gou)

In [19]:
hover = HoverTool(
            tooltips=[
                ('(x,y)','(@x,@y)')
            ]
                )

p = figure(title='合成ポジション',plot_width=500, plot_height=300, tools=[hover])
p.line(g.index, g)

show(p)

* 動いたら大体儲かるやん!
## 当日やったらな

In [20]:
gou2 = {}

for i in range(14000,22000):
    call = iv.prem_call(i,k,r,q,10/365, sigma)
    fut = 0.18 * (s-i)
    posi = call + fut
    
    gou2[i] = posi
    
g2 = pd.Series(gou2)

In [21]:
hover = HoverTool(
            tooltips=[
                ('(x,y)','(@x,@y)')
            ]
                )

p = figure(title='合成ポジション', plot_width = 500, plot_height=300,tools=[hover])
p.line(g.index, g, color='blue', legend='30days')
p.line(g2.index, g2, color='green', legend='10days')

p.legend.location='top_left'
show(p)